In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint

from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

# import pandas as pd
# from pandas import DataFrame
# from sklearn import linear_model
# import statsmodels.api as sm

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

In [2]:
from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

In [ ]:
def test_intersection(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)

In [3]:
from scripts.UVMappingPipeline import UVMappingPipeline
from scripts.VBOATS import VBOATS

pipe = UVMappingPipeline()
vboat = VBOATS()

vboat.testMaxGndStep = 16

In [ ]:
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_1")
found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/test_keyParams")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_1")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/test_cases")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion/huh")

sFImgs = sorted(found_imgs)

In [ ]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))

lblSz1 = widgets.Label(value="Deadzone Sizes (U,V):", layout=layCenter)
sz1Selector = widgets.IntSlider(value=0,min=0,max=20,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=3,min=0,max=20,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Strip[0] Deadzone Sizes (U,V):", layout=layCenter)
sz3Selector = widgets.IntSlider(value=4,min=0,max=20,step=1,description=' ', layout = layStretch)
sz4Selector = widgets.IntSlider(value=5,min=0,max=20,step=1,description=' ', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=True, description='Use Test Pipeline', layout = layCenter)
flag2 = widgets.Checkbox(value=False, description='Show Computational Timing', layout = layCenter)
flag3 = widgets.Checkbox(value=False, description='Use Morphs', layout = layCenter)

In [ ]:
def thresholding(_img, dx=0,dy=3, sz1=5,sz2=10,tu0=0.15,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True, flag_timing=False,flag_morph=True
):
#     global overlay
    
    if(flag_test):
        vboat.dW = sz2;        vboat.dH = sz1
        vboat.dead_x = dx;     vboat.dead_y = dy
        vboat.dThs = [tdu,tdu2,win1thr,tdv]
        vboat.dHplus = win1
        
        vboat.kszs[0] = (k11,k12)
        vboat.kszs[1] = (k21,k22)
        vboat.kszs[2] = (k31,k32)
        vboat.testLowCntThresh = k21
        vboat.testHighCntThresh = k31
        vboat.testRestThreshRatio = tdu
        vboat.flag_morphs = flag_morph
        
        vboat.vmap_search_window_height = searchWin
        
        vboat.pipelineTest(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)
        display_obstacles = cv2.applyColorMap(display_obstacles,cv2.COLORMAP_JET)
        
        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(255,255,255),1)
        
        overlay = make_uv_overlay(display_obstacles,vboat.umap_deadzoned,vboat.vmap_raw)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
        pplot(overlay,'Overlay')

        dispU1,dispU2,compU = vboat.umap_displays()
        dispV1,dispV2,compV = vboat.vmap_displays()

        dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
        dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
        pplot(dispU2,'Umap Display')
        pplot(dispV2,'Vmap Display')
        pplot(vboat.vmap_processed,"Processed Vmap")
        
        test_vstrip_filter()
#         _,strip, nStrip,_, disp = vboat.test_filter_first_umap_strip(vboat.stripsU_raw[0],51,[1,1,1,1,1,1], ratio_thresh=vboat.testRestThreshRatio)
#         pplots([strip,nStrip,disp],"Compare",(3,1))
        
        distances = []
        angles = []
        umap = vboat.umap_raw
        xs = vboat.xBounds
        ds = np.array(vboat.dbounds)
        obs = vboat.obstacles
        nObs = len(ds)
        if(nObs is not 0):
            for i in range(nObs):
                disparities = ds[i]
                us = [obs[i][0][0], obs[i][1][0]]
                vs = [obs[i][0][1], obs[i][1][1]]
                z,ux,uy,uz = vboat.calculate_distance(umap,us,disparities,vs)

                theta = math.acos((uz/z))

                distances.append(z)
                angles.append(theta)
                print("Obstacle [%d] Distance, Angle: %.3f, %.3f" % (i,z,np.rad2deg(theta)))
        
    else:
        vboat.pipeline(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        pplot(display_obstacles,'Pipeline Obstacles')

In [ ]:
%reload_ext autoreload
plt.close('all')

p = interactive(thresholding,
    _img = imgPathSelector, dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_morph = flag3
)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-4], p.children[-3], p.children[-2]], layout = layCenter))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
display(tabs)

In [ ]:
vboat.debug_obstacle_search = False
vboat.debug = False

In [ ]:
print(vboat.dbounds)
pplot(vboat.umap_raw,0)
print(np.uint16.argmax())

In [ ]:
img = cv2.imread("/home/hyoung/data/assured_autonomy/heightmap_test/HM.png", cv2.IMREAD_GRAYSCALE)
h,w = img.shape[:2]
gain = 65535/float(np.max(img))
tmp = np.float32(img / float(255))
newImg = np.uint16(tmp*65535)
print(gain)
noise_img = np.random.rayleigh(scale=1, size=(h, h))
noise_img = (noise_img * 65535) / float(np.max(noise_img))
noise_img = np.uint16(noise_img*0.1)
pplot(noise_img,"noise_img")
pplots([img,newImg+noise_img],"img",(1,2))

compImg = newImg+noise_img
compImg = compImg.astype(np.uint16)
cv2.imwrite('/home/hyoung/data/tmpHM.png',compImg)


test = cv2.imread('/home/hyoung/data/tmpHM.png',cv2.IMREAD_UNCHANGED)
print(test.dtype)
pplot(test,"test")

In [ ]:
def test_vstrip_filter():
    vmap = np.copy(vboat.vmap_raw)
    vmax = np.max(vmap)

    testStrips = []
    stripsPv = []
    threshsV = [0, 70, 40,60,60]
    nThreshsV = int(math.ceil((vboat.dmax/256.0) * len(threshsV)))


    stripsV = strip_image(vboat.vmap_deadzoned, nstrips=nThreshsV, horizontal_strips=False)
    pplots(stripsV,"Raw Vmap Strips",(1,nThreshsV))

    # RAW VStrip 1
    testStrips.append(stripsV[0])

    stripMax = np.max(stripsV[0])
    relRatio = stripMax/float(vmax)


    for i, strip in enumerate(stripsV):
        _, tmpStrip = cv2.threshold(strip, threshsV[i], 255, cv2.THRESH_TOZERO)
        stripsPv.append(tmpStrip)

#     pplots(stripsPv,"Pre-Filtered Vmap Strips",(1,nThreshsV))

    vsThs = np.array([
        [0.25, 0.0],
        [0.5, 0.0]
    ])

    # Filtered VStrip 1 STEP 0
    testStrips.append(np.copy(stripsPv[0]))

    tmpMax = np.max(stripsPv[0])

    # Filtered VStrip 1 STEP 1
    testStrips.append(np.copy(stripsPv[0]))

    tmpStrip = stripsPv[0]
    hs, ws = tmpStrip.shape[:2]

    tmpdead = tmpStrip[:, 0:3]
    tmprest = tmpStrip[:, 3:ws]

    deadMax = np.max(tmpdead)
    restMax = np.max(tmprest)
    dmRatio = deadMax / float(stripMax)
    rmRatio = restMax/float(stripMax)
    print("stripMax,vmapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,vmax,deadMax,restMax))
    print("relRatio, dmRatio,rmRatio: %.3f, %.3f, %.3f" % (relRatio, dmRatio,rmRatio))
#     pplots([tmpdead,tmprest], "Pre-CLAHE Vmap", (1,2))

    # _, fa = cv2.threshold(tmpdead, tfa,255,cv2.THRESH_TOZERO) #THRESH_TOZERO

    tCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(5,1))
    tCLAH2 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs,3))
    dCl = tCLAH.apply(tmpdead)
    dCl2 = tCLAH2.apply(tmprest)

#     pplots([dCl,dCl2],"DCLs V",(1,2))
    pc1 = dCl[:, 0:1]
    pc2 = dCl[:, 1:dCl.shape[1]]

    # _,pc1 = cv2.threshold(pc1, 200,255,cv2.THRESH_TOZERO)
    _,pc1 = cv2.threshold(pc1, 150,255,cv2.THRESH_TOZERO)
    _,pc2 = cv2.threshold(pc2, 0,255,cv2.THRESH_TOZERO)
    tmpdead = np.concatenate((pc1,pc2), axis=1)

#     pplots([pc1,pc2],"dCl strips",(1,2))


    cldMax = np.max(tmpdead)
    clrMax = np.max(dCl2)



    # deadThreshGain = (stripMax - deadMax)/255.0
    # deadThreshOffset = int(base_thresh*deadThreshGain)
    # if not vboat.is_ground_present: deadThresh = deadThreshOffset
    # else: deadThresh = base_thresh + deadThreshOffset

    # dead_strip[dead_strip < deadThresh] = 0
    # deadMax = np.max(dead_strip)


    deadMaxGain = (cldMax - deadMax)/float(cldMax)
    restMaxGain = (clrMax - restMax)/float(clrMax)
    print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
    newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
    print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))

    ratio_thresh = 0.25;   base = 64
    g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
    t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
    t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
    t31 = int((g2+g2*restMaxGain)*clrMax)
    print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
    print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
    print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

    _,tmprest = cv2.threshold(dCl2, t31,255,cv2.THRESH_TOZERO)
    stripsPv[0] = np.concatenate((tmpdead,tmprest), axis=1)


    # Filtered VStrip 1 STEP 2
    testStrips.append(np.copy(stripsPv[0]))

    dw = 5;        vdTh = [0.15, 0]

    # dead_strip = stripsPv[0][:, 0:dw]
    # rest_strip = stripsPv[0][:,dw:stripsPv[0].shape[1]]

    # print("[INFO] Shapes stripsPv[0], dead_strip, rest_strip: %s, %s, %s" % (str(stripsPv[0].shape), str(dead_strip.shape),str(rest_strip.shape)))

    # dead_strip[dead_strip < tmpMax*vdTh[0]] = 0
    # rest_strip[rest_strip < tmpMax*vdTh[1]] = 0
    # stripsPv[0] = np.concatenate((dead_strip,rest_strip), axis=1)

    # # Filtered VStrip 1 STEP 3
    # testStrips.append(np.copy(stripsPv[0]))

    # print("[INFO] stripsPv[0] shape: %s" % (str(stripsPv[0].shape)))

    # stripsPv[1][stripsPv[1] < np.max(stripsPv[1]*0.9)] = 0
    tmpMax = np.max(stripsPv[1])
    tH = stripsPv[1].shape[0]
    top_strip = stripsPv[1][0:tH/3, :]
    bot_strip = stripsPv[1][tH/3:tH, :]

    top_strip[top_strip < tmpMax*vsThs[1,0]] = 0
    bot_strip[bot_strip < tmpMax*vsThs[1,1]] = 0
    stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)

    newV = np.concatenate(stripsPv, axis=1)

#     pplots(testStrips,"1st Vmap Strip Filter Steps",(1,len(testStrips)))

    tmp = np.copy(newV)
    vmap = cv2.bitwise_and(tmp,tmp,mask=cv2.bitwise_not(vboat.vmask))
    pplot(vmap,"New Vmap")
    obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, vboat.disparityBounds, vboat.xBounds, ground_detected=vboat.is_ground_present,verbose=True)

    dispV = cv2.applyColorMap(np.copy(vmap),cv2.COLORMAP_JET)
    for wins in windows:
        for win in wins:
            cv2.rectangle(dispV,win[0],win[1],(0,255,255), 1)
    dispV = cv2.cvtColor(dispV,cv2.COLOR_BGR2RGB)
    pplot(dispV,"dispV")


    displayObs = cv2.cvtColor(np.copy(vboat.img), cv2.COLOR_GRAY2BGR)
    displayObs = cv2.applyColorMap(displayObs,cv2.COLORMAP_JET)

    for ob in obs:
        cv2.rectangle(displayObs,ob[0],ob[1],(255,255,255),1)


    overlay = cv2.cvtColor(displayObs,cv2.COLOR_BGR2RGB)
    pplot(overlay,'Test Obs')

In [ ]:
print(windows[0])

In [ ]:
verbose = True;     n = 3
umap = np.copy(vboat.umap_raw);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[0])
# Stripping
hs, ws = strip.shape[:2];              dh = hs / n
print("strip.shape: ",strip.shape[:2])
# pplot(strip,"strip")
dead_strip = strip[0:dh, :];           oDead = np.copy(dead_strip)
rest_strip = strip[dh:hs, :];          oRest = np.copy(rest_strip)
print("Shapes -- deadStrip, restStrip: %s, %s" % (str(dead_strip.shape),str(rest_strip.shape)))

hd, wd = dead_strip.shape[:2]
hr, wr = rest_strip.shape[:2]

umapMax = np.max(umap)
stripMax = np.max(strip)
restMax = np.max(rest_strip)

ksz2 = (75,2)
ksz1 = (30,1)
kernelr = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,ksz2)

tfa = int(0.5*umapMax);
print("tfa: %d" %(tfa))

tmpdead = dead_strip[0:3, :]
tmprest = dead_strip[3:hd, :]
pplot(tmpdead,"pre TEST")
pplot(tmprest,"pre REST")
tCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(1,20))
dCl = tCLAH.apply(tmpdead)

print("dCl Shape: %s" % str(dCl.shape))


pc1 = dCl[0:1, :]
pc2 = dCl[1:dCl.shape[0], :]
pplots([pc1,pc2],"dCl strips",(2,1))
print("[DEBUG] test_filter_first_umap_strip() -----  Shapes: pc1 [%s], pc2 [%s]" % (str(pc1.shape),str(pc2.shape)))


_,pc1 = cv2.threshold(pc1, 256,255,cv2.THRESH_TOZERO)
_,pc2 = cv2.threshold(pc2, 175,255,cv2.THRESH_TOZERO)

tmpdead2 = np.concatenate((pc1,pc2), axis=0);

_,tmprest = cv2.threshold(tmprest, 55,255,cv2.THRESH_TOZERO)

# pplots([dCl,dCl3],"dCl3",size=(2,1),flag_resize=True,scale=(5,1))
# pplot(tmpdead2,"TEST")
# pplot(tmprest,"rest")

_, fa = cv2.threshold(tmpdead, tfa,255,cv2.THRESH_TOZERO) #THRESH_TOZERO

mask = cv2.morphologyEx(fa, cv2.MORPH_CLOSE, kernelD)
tmpdead2 = cv2.morphologyEx(tmpdead2, cv2.MORPH_CLOSE, kernelr)
tmprest = cv2.morphologyEx(tmprest, cv2.MORPH_CLOSE, kernelD)
# pplot(tmpdead2,"post TEST")
# pplot(tmprest,"ppost rest")
dead_strip = np.concatenate((tmpdead2,tmprest), axis=0);
# dead_strip = np.concatenate((fa,tmprest), axis=0);
# dead_strip = np.concatenate((mask,tmprest), axis=0);
# pplot(dead_strip,"dead_strip")

oStrip = np.concatenate((dead_strip,rest_strip), axis=0);
oDead2 = np.copy(dead_strip)

# pplots([oDead,newdead,trip],"oDead", (3,1))
pplot(oStrip,"oStrip")

# stripMax = np.max(oStrip)
deadMax = np.max(dead_strip)

# Initial Stats
relRatio = stripMax/float(umapMax)
dmRatio = deadMax / float(stripMax)
rmRatio = restMax/float(stripMax)
print("relRatio, dmRatio,rmRatio: %.3f, %.3f, %.3f" % (relRatio, dmRatio,rmRatio))
# Deadzone Filtering
threshD = 50;         base_thresh = 35
deadThreshGain = (stripMax - deadMax)/255.0;        deadThreshOffset = int(base_thresh*deadThreshGain)
if not vboat.is_ground_present: deadThresh = deadThreshOffset
else: deadThresh = base_thresh + deadThreshOffset
dead_strip[dead_strip < deadThresh] = 0
# dead_strip[dead_strip < deadThresh] = 0
if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))
deadMax = np.max(dead_strip)
# Perform CLAHE

# claheDead = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hd,wd))
claheDead = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hd,wd))
claheRest = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hr/2,wr))
clD = claheDead.apply(dead_strip);         copyCld = np.copy(clD)
clR = claheRest.apply(rest_strip);         copyClr = np.copy(clR)
_,stripD = cv2.threshold(clD, threshD,255,cv2.THRESH_TOZERO)                # Filter CLAHE Deadzone
cldMax = np.max(stripD);    clrMax = np.max(clR)                            # CLAHE Maxes
if deadMax == 0: newDeadMax = 1
else: newDeadMax = deadMax
deadMaxGain = (cldMax - deadMax)/float(cldMax)
restMaxGain = (clrMax - restMax)/float(clrMax)
newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))
# Remainder Filtering
ratio_thresh = 0.25;   base = 64
print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
t31 = int((g2+g2*restMaxGain)*clrMax)
print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
_,stripR = cv2.threshold(clR, t31,255,cv2.THRESH_TOZERO)
# _,stripR = cv2.threshold(clR, 43,255,cv2.THRESH_TOZERO)

# pplot(stripD,"stripD")
# pplot(stripR,"stripR")

# Morphological Transformations
newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# Find Contours
contours,_ = vboat.find_contours(pStrip, 50.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
pplots([strip,newStrip,disp],"Test Compare",(3,1))

In [ ]:
pplot(umap,"umap")
print(umap.shape[0]/2)

In [ ]:
dm = vboat.dmax
pd = vboat.dmax/256.0
pd = 100/256.0
tmpN = pd*6
print(pd)
print(tmpN)
tmp = math.ceil(tmpN)
print(tmp)

In [ ]:
del(display)

In [ ]:
verbose = True;     n = 6
umap = np.copy(vboat.umap_deadzoned)
stripsU = strip_image(umap, nstrips=n)
strip = np.copy(stripsU[1])
tstrip = np.copy(strip)

hs, ws = strip.shape[:2];               dh = hs / n;    umapMax = np.max(umap);     stripMax = np.max(strip)
relRatio = stripMax/float(umapMax)
threshD = int(0.25*stripMax)
print("relRatio, stripMax,threshD: %.3f, %d, %d" % (relRatio,stripMax,threshD))

# if not vboat.is_ground_present: deadThresh = deadThreshOffset
# else: deadThresh = base_thresh + deadThreshOffset

tstrip[tstrip < threshD] = 0
# dead_strip[dead_strip < deadThresh] = 0

# if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))

# Perform CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs,ws))
clD = clahe.apply(strip);         copyCld = np.copy(clD)
# _,stripD = cv2.threshold(clD, threshD,255,cv2.THRESH_TOZERO)
cldMax = np.max(clD)
# if deadMax == 0: newDeadMax = 1
# else: newDeadMax = deadMax
deadMaxGain = (cldMax - stripMax)/float(cldMax)
newStripMax = int(stripMax*deadMaxGain)
print("newStripMax: %d" % (newStripMax))
# Remainder Filtering
ratio_thresh = 0.25;   base = 64
# print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
# print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
g1 = deadMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
t1 = int(g1*cldMax);      t2 = int(g2*cldMax);      t3 = int(g3*cldMax);      t4 = int(g4*cldMax)
t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
t31 = int(g1*t2);
print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
# _,stripR = cv2.threshold(clR, t2-t31,255,cv2.THRESH_TOZERO)
_,newStrip = cv2.threshold(clD, 90,255,cv2.THRESH_TOZERO)

# pplot(stripR,"stripR")

# Morphological Transformations
ksize = (5,1)
# newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# Find Contours
contours = vboat.find_contours(pStrip, 50.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([strip,tstrip,newStrip,disp],"Sections U Strip 1",(4,1))
pplots([strip,newStrip,disp],"Test Compare UStrip 1",(3,1))

In [ ]:
verbose = False;     n = 6
vmap = np.copy(vboat.vmap_raw)
slab = np.ones((vmap.shape[0],1),dtype=np.uint8)*255

# tmpV = cv2.cvtColor(vmap, cv2.COLOR_GRAY2BGR)
mask_inv = cv2.bitwise_not(vboat.vmask)
newMask = np.concatenate((mask_inv,slab), axis=1)
# pplot(vmap,"vmap")
# pplot(newMask,"newMask")
print(vmap.shape,newMask.shape)
vmap = cv2.bitwise_and(vmap,vmap,mask=newMask)
stripsV = strip_image(vmap, nstrips=n, horizontal_strips=False);
strips = strip_image(stripsV[0], nstrips=n, horizontal_strips=False);
stripv = np.copy(stripsV[0])
pplots(stripsV,"vstrips",size=(1,6))
# Stripping
hs, ws = stripv.shape[:2];              dh = hs / n
print("hs, ws: %d, %d" % ( hs, ws))
# Initial Stats
vmapMax = np.max(vmap); stripMax = np.max(stripv)
relRatio = stripMax/float(vmapMax)

print("vmapMax, stripMax, relRatio: %d, %d, %.3f" % (vmapMax, stripMax, relRatio))
# Deadzone Filtering
threshD = 7;         base_thresh = 35
# deadThreshGain = (stripMax - deadMax)/255.0;        deadThreshOffset = int(base_thresh*deadThreshGain)
# deadThresh = base_thresh + deadThreshOffset;        dead_strip[dead_strip < deadThresh] = 0
# if verbose: print("[Deadzone Filtering] base, gain, offset, thresh: %d, %.2f, %d, %d" % (base_thresh,deadThreshGain,deadThreshOffset,deadThresh))
# deadMax = np.max(dead_strip)

stripv[stripv < 1] = 0
pplot(stripv,"stripv")
# Perform CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs/6,ws/6))
cl = clahe.apply(stripv);         copyCl = np.copy(cl)
clMax = np.max(cl)
_,stripF = cv2.threshold(cl, 64,255,cv2.THRESH_TOZERO)                # Filter CLAHE Deadzone

# if deadMax == 0: newDeadMax = 1
# else: newDeadMax = deadMax
# deadMaxGain = (cldMax - deadMax)/float(cldMax)
# restMaxGain = (clrMax - restMax)/float(clrMax)
# newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
# print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))
# # Remainder Filtering
# ratio_thresh = 0.25;   base = 64
# print("stripMax,umapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,umapMax,deadMax,restMax))
# print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
# g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
# t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
# t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
# t31 = int(g1*t2);
# print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
# print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
# print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

# # _,stripR = cv2.threshold(clR, threshR,255,cv2.THRESH_TOZERO)
# _,stripF = cv2.threshold(clR, t2-t31,255,cv2.THRESH_TOZERO)
# # _,stripR = cv2.threshold(clR, 43,255,cv2.THRESH_TOZERO)



# Morphological Transformations
# newStrip = np.concatenate((stripD,stripR), axis=0); ksize = (5,1)
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize); pStrip = cv2.morphologyEx(newStrip, cv2.MORPH_CLOSE, kernel)
# # Find Contours
# contours = vboat.find_contours(pStrip, 50.0)
# disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
# [cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
# Plots
pplots([stripv,cl,stripF],"CLAHE V Sections",(1,3))
# pplots([strip,newStrip,disp],"Test Compare",(3,1))

In [ ]:
# display(Math(r'$$f(a) = \lim_{{x \to a}} \frac{{f(x) - f(a)}}{{x - a}} = {}$$'))
# r1 = (r'$$relRatio = \frac{{stripMax}}{{umapMax}} = {}$$'.format(relRatio))
# r2 = (r'$$dmRatio = \frac{{deadMax}}{{stripMax}} = {}$$'.format(dmRatio))
# r3 = (r'$$restMax = \frac{{restMax}}{{stripMax}} = {}$$'.format(rmRatio))
# tmp = Math(r'{}{}{}'.format(r1,r2,r3))
tmp = Math(r'$$relRatio = \frac{{strip}}{{umap}} =\frac{a}{b}= {}\qquad dmRatio = \frac{{deadzone}}{{strip}} = \frac{}{} = {}\qquad rmRatio = \frac{{rest}}{{strip}} = \frac{}{} = {}$$'.format(a=stripMax,b=umapMax,round(relRatio,3),deadMax,stripMax,round(dmRatio,3),restMax,stripMax,round(rmRatio,3)))
display(tmp)
# display(r1)
# display(r2)
# display(r3)
# r4 = Math(r'$$rmRatio = \frac{{stripMax}}{{umapMax}} = {}$$')

In [ ]:
ustr = np.copy(stripsU[0])
tmpMax = np.max(ustr)

dh = ustr.shape[0]/ 6
drest = (ustr.shape[0]-dh)

dead_strip = ustr[0:dh, :]
orig = np.copy(dead_strip)
rest_strip = ustr[dh:ustr.shape[0], :]

rMax = np.max(dead_strip)

baseDeadThresh = 35
deadThreshGain = (tmpMax - rMax)/255.0
deadThreshOffset = int(baseDeadThresh*deadThreshGain)
deadThresh = baseDeadThresh + deadThreshOffset

# print("base, gain, offset, thresh: %d, %.2f, %d, %d" % (baseDeadThresh,deadThreshGain,deadThreshOffset,deadThresh))

dead_strip[dead_strip < deadThresh] = 0
dClahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(dead_strip.shape[0],dead_strip.shape[1]))
dcl = dClahe.apply(dead_strip)
dcl2 = np.copy(dcl)
dcl2[dcl2 < 7] = 0
pplots([orig,dead_strip,dcl,dcl2],"Test",(4,1))

hs,ws = rest_strip.shape[:2]

timg2 = np.copy(rest_strip)
clahe2 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(timg2.shape[0],timg2.shape[1]))
cl2 = clahe2.apply(timg2)
testMax2 = np.max(cl2)
testThresh2 = int(0.5 * testMax2)

# print("testMax2, testThresh2: %d, %d" % (testMax2,testThresh2))

_,testStrip2 = cv2.threshold(cl2, testThresh2,255,cv2.THRESH_TOZERO)
newStrip2 = np.concatenate((dcl2,testStrip2), axis=0)
pplots([stripsU[0],newStrip2],"CLAHE Compare",(2,1))

In [ ]:
print(params[0].keys())
print(params[0]['absMaxs'][1])

In [ ]:
np.set_printoptions(linewidth=90)
gs = []; ofs = []; vs = []; errs = []; ts=[]
rms = []; ms = []
for p in params:
    tmpSm = p['relMaxs'][0]
    tmpM = p['absMaxs'][1]
    trueThresh = p['threshs'][0]
    
    tmpG = (tmpM - tmpSm)/255.0
#     tmpOf = int(tmpG*tmpSm)
    tmpOf = int(tmpG*b)
    tmpV = b + tmpOf
    err = trueThresh-tmpV
    
    rms.append(tmpSm)
    ms.append(tmpM)
    ts.append(trueThresh)
    gs.append(tmpG)
    ofs.append(tmpOf)
    vs.append(tmpV)
    errs.append(err)

ms = np.array(ms,dtype=int)
rms = np.array(rms,dtype=int)
errs = np.array(errs,dtype=int)
vs = np.array(vs,dtype=int)
ts = np.array(ts,dtype=int)
# print(errs)
# print(rms.tolist())
# print(vs.tolist())
# print(ts.tolist())

plist("Absmax:\t",ms.tolist())
plist("max:\t",rms.tolist())
plist("est:\t",vs.tolist())
plist("true:\t",ts.tolist())

# plist("threshs: ", vs, sep=',')
# plist("errors: ", errs)

In [ ]:
m = 193
sm = 114
b = 35
g = m - sm
g = g/255.0

of = int(g*b)
v = b + of

print(g,of,v)

In [ ]:
threshes = [5, 5, 5, 1,4,5]

tmpdict = {
    "file": imgPathSelector.value,
    "absMaxs" : absMaxs,
    "relMaxs": relMaxs,
    "avgVals": avgVals,
    "threshs": threshes
}
pp.pprint(tmpdict)
keyParams.append(tmpdict)

In [ ]:
# keyParams = []

In [ ]:
svFile = "/home/hunter/data/vboats/test_keyParams/keyParams.pkl"

In [ ]:
svFile = "/home/hunter/data/vboats/test_keyParams/keyParams.pkl"
save_pickle("/home/hunter/data/vboats/test_keyParams/keyParams.pkl",keyParams)

In [ ]:
params = pickle.load(open(svFile, "rb" ))
# print(len(params))
newParams = [d for d in params]
# print(len(newParams))

In [ ]:
tmpParams = pickle.load(open("/home/hunter/data/vboats/test_keyParams/modThreshParams.pkl", "rb" ))

In [ ]:
i = 19
newStrips = []
# pp.pprint(params[i])
threshsU = tmpParams[i]['threshs']
relMaxs = tmpParams[i]['relMaxs']
img_path = tmpParams[i]['file']
plist("threshsU: ",threshsU)
plist("relMaxs: ",relMaxs)
n = len(threshsU); dead_y = 3;
print(i)

# threshsU[0] = 5
# threshsU[1] = 14
# threshsU[2] = 3
# threshsU[3] = 11
# threshsU[4] = 5
# threshsU[5] = 11


img = vboat.read_image(img_path)
pplot(img,"Img")
raw_umap, _, _ = vboat.get_uv_map(img)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)

stripsU = strip_image(raw_umap, nstrips=n)
strips = strip_image(stripsU[0], nstrips=n)

for i, strip in enumerate(strips):
    tmpThresh = threshsU[i]
    threshs.append(tmpThresh)
    _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_TOZERO)  
    newStrips.append(tmpStrip)
    
newUmap = np.concatenate(newStrips, axis=0)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
pUmap = cv2.morphologyEx(newUmap, cv2.MORPH_CLOSE, kernel)

pplots(newStrips,"Thresholded Umap 0", flag_resize=True,scale=(5,1))
pplots(strips,"Raw Umap 0", flag_resize=True,scale=(5,1))
pplots([stripsU[0],newUmap],"Umap Compare",(2,1))

contours = vboat.find_contours(newUmap, 25.0)
disp = cv2.applyColorMap(newUmap,cv2.COLORMAP_PARULA)
disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
pplot(disp,"Umap")

In [ ]:
params1 = pickle.load(open("/home/hunter/data/vboats/test_keyParams/modThreshParams.pkl", "rb" ))
params2 = pickle.load(open("/home/hunter/data/vboats/test_keyParams/keyParams.pkl", "rb" ))
for i in range(len(params1)):
#     print(len(params1[i]),len(params2[i]))
    print(params1[i]['file']+"\r\n"+params2[i]['file'])
    print

In [ ]:
newParams[0]['threshs'][0] = 60
newParams[0]['threshs'][1] = 16

newParams[2]['threshs'][1] = 14

newParams[3]['threshs'][2] = 12
newParams[3]['threshs'][3] = 10
newParams[3]['threshs'][4] = 13
newParams[3]['threshs'][5] = 12

newParams[4]['threshs'][0] = 33
newParams[4]['threshs'][4] = 7
newParams[4]['threshs'][5] = 11

newParams[5]['threshs'][0] = 35

newParams[6]['threshs'][3] = 18
newParams[6]['threshs'][4] = 12

newParams[12]['threshs'][4] = 3

newParams[14]['threshs'][2] = 3

newParams[15]['threshs'][3] = 7
newParams[15]['threshs'][4] = 5

newParams[16]['threshs'][2] = 3
newParams[16]['threshs'][3] = 11

newParams[17]['threshs'][4] = 5

newParams[20]['threshs'][0] = 5

# pp.pprint(params[i])

In [ ]:
newSvFile = "/home/hunter/data/vboats/test_keyParams/modifiedThreshParams.pkl"
f = open(newSvFile,"wb")
pickle.dump(newParams,f)
f.close()

In [ ]:
testParams = [d for d in tmpParams]
for i, d in enumerate(tmpParams):
    newFile = "params_test_frame_" + str(i) + ".png"
    new_name = "/home/hunter/data/vboats/test_keyParams/" + newFile
    img_path = d['file']
#     tmp = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     cv2.imwrite(new_name,tmp)
    testParams[i]['file'] = newFile
    
f = open("/home/hunter/data/vboats/test_keyParams/keyParams.pkl","wb")
pickle.dump(testParams,f)
f.close()

In [ ]:
newParams = []

In [ ]:
# count = 0
# done = False
# while(not done):
#     data = getpass.getpass("")
#     sys.stdout.flush()
#     if data == 'o': count +=1
#     elif data == 'p': count -=1
#     elif data == 'q': done = True

In [ ]:
N = len(baseThreshs)
tests = ['Abs Max', 'Major Strip Max', 'Minor Strip Max']

plist("absMaxs:\t",absMaxs)
print
plist("baseThreshs:\t",baseThreshs,sep=',\t')
plist("absRatios:\t",absRatios,sep=',\t')
plist("relMaxs:\t",relMaxs,sep=',\t')
plist("avgVals:\t",avgVals,sep=',\t')
print

absMax = absMaxs[0]; strMax = absMaxs[1]

relRatios = []; relRatios2 = []; avgRatios = []; avgRatios2 = [];
ratios = []; maxs = []; gains = [];
dratio = dmax/255.0
for i in range(N):
    maxs.append([absMax, strMax,relMaxs[i]])
    relRatio = (absMax-relMaxs[i])/float(absMax)
    relRatio2 = (strMax-relMaxs[i])/float(strMax)
    avgRatio = (absMax-avgVals[i])/float(absMax)
    avgRatio2 = (strMax-avgVals[i])/float(strMax)
    
    relRatios.append(relRatio)
    relRatios2.append(relRatio2)
    avgRatios.append(avgRatio)
    avgRatios2.append(avgRatio2)
    
    ratios.append([relRatio,relRatio2,avgRatio,avgRatio2,1-relRatio,1-relRatio2,1-avgRatio,1-avgRatio2])
maxs = np.array(maxs)
_,nMaxs = maxs.shape[:2]
ratios = np.array(ratios).T
ntests,nstrips = ratios.shape[:2]

ratio_lbls = ["Ratios "+str(t) for t in range(ntests)]
gain_lbls = ["Gains "+str(t) for t in range(ntests)]
thresh_lbls = ["Thresholds "+str(t) for t in range(ntests)]

test_lbls = ["Test: "+str(test) for test in tests]
col_lbls = ["Strip "+str(strip) for strip in range(nstrips)]
val_lbls = [str(test) for test in tests]

threshs = np.zeros((ntests,nstrips,nMaxs), dtype=np.uint8)
for ch in range(nMaxs):
    for test in range(ntests):
        tmpGains = []
        for strip in range(nstrips):
            tmpGain = baseThreshs[strip]*ratios[test,strip]
            tmpGains.append(tmpGain)
            tmpThresh = tmpGain * maxs[strip,ch]
            threshs[test,strip,ch] = tmpThresh
        if ch == 0: gains.append(tmpGains) 
gains = np.array(gains)

pmat(maxs,val_lbls,col_lbls)
pmat(ratios,col_lbls,ratio_lbls)
pmat(gains,col_lbls,gain_lbls)#,"\r\nGains")

for i, lbl in enumerate(test_lbls):
    tThreshs = threshs[:,:,i]
    pmat(tThreshs,col_lbls,thresh_lbls,lbl)

In [ ]:
pklParentDir = "/home/hyoung/data/vboats/test_keyParams"
pklFile = os.path.join(pklParentDir,"keyParams.pkl")
params = load_pickle(pklFile)
print(params[0].keys())
# pp.pprint([p['relMaxs'][0] for p in params])

In [ ]:
# pp.pprint(params)
tmp = params[0]
tmp['relMaxs']
# df = DataFrame(tmp,columns=tmp.keys())
# df

In [ ]:
temp = vboat.umap_deadzoned
tempv = vboat.vmap_raw

tmpStrs = strip_image(temp, nstrips=6)
tmp0Strs = strip_image(tmpStrs[0], nstrips=6)

tmpStrsv = strip_image(tempv, nstrips=6,horizontal_strips=False)
tmpv0Strs = strip_image(tmpStrsv[0], nstrips=6,horizontal_strips=False)

pplots(tmp0Strs,"Umap 0", flag_resize=True,scale=(5,1))
pplots(tmpv0Strs,"vmap strip",(1,6),flag_resize=True,scale=(1,10))
pplots(tmpStrs,"umap strips")

tmps = [vboat.umap_deadzoned,vboat.umap_processed]
pplots(tmps,"umaps",(len(tmps),1))

In [ ]:
print(vboat.umap_deadzoned.shape,vboat.umap_processed.shape)

In [ ]:
cnts = np.array([cv2.arcLength(cnt,True) for cnt in vboat.contours])
print(np.unique(cnts))

In [ ]:
tmpmax = dmax
# rat = tmpmax/255.0
tmpt = threshsU[0]

rat = 187/255.0

gain = tmpt*rat*10
print(tmpmax,rat, gain,gain*61)

In [ ]:
hm = cv2.imread("/home/hyoung/data/HM.png", cv2.IMREAD_GRAYSCALE)
noise_img = cv2.imread("/home/hyoung/data/R.png", cv2.IMREAD_GRAYSCALE)
pplot(hm,"raw HM")
pplot(noise_img,"noise")
scaled = noise_img * 0.1
pplot(scaled,"scaled noise")
composite_heightmap = hm + noise_img
composite_heightmap_scaled = hm + scaled

print(noise_img[0,0],scaled[0,0])

pplot(composite_heightmap,"comp")
cv2.imwrite("/home/hyoung/data/composite.png", composite_heightmap)
cv2.imwrite("/home/hyoung/data/composite_scaled.png", composite_heightmap_scaled)
cv2.imwrite("/home/hyoung/data/scaled_nosie.png", scaled)

In [ ]:
tImg = np.copy(vboat.img)
umap = vboat.umap_raw
vmap = vboat.vmap_raw
deadu = vboat.umap_deadzoned
pv = vboat.vmap_processed
pu = vboat.umap_processed

# pplot(tImg,'Input')
pplot(umap,'Raw Umap')
# pplot(vmap,'Raw Vmap')
pplot(deadu,'Deadzoned Umap')
pplot(pv,'Processed vmap')



# mn = np.mean(tImg)
# st = np.std(tImg[tImg > 10])
# norm = (tImg - mn)/st

# norm = np.expand_dims(norm,axis=-1)

# norm = cv2.normalize(norm,None,0.0,255.0,cv2.NORM_MINMAX)
# norm = np.asarray(norm, dtype=np.uint8)
# nUmap, nVmap,_ = vboat.get_uv_map(norm)

# sub = nUmap - umap 
# pplot(sub,'Sub')

# copy = vmap.copy()
# # copy = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)
# copy = np.float32(copy)
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# K = 150
# ret,label,center=cv2.kmeans(copy,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# # Now convert back into uint8, and make original image
# center = np.uint8(center)
# res = center[label.flatten()]
# res2 = res.reshape((vmap.shape))
# pplot(res2,'kmeans')

# nUmap = nUmap**2
# nUmap[nUmap < np.max(nUmap*0.3)] = 0

# pplot(nUmap,'Norm Umap')

# pplot(vboat.umap_processed,'Processed Umap')

# _, tmpU = cv2.threshold(vmap, 10, 255,cv2.THRESH_TOZERO)
# tmpU = cv2.equalizeHist(tmpU)

# pplot(tmpU,1)

In [ ]:
dispU1,dispU2,compU = vboat.umap_displays()
dispV1,dispV2,compV = vboat.vmap_displays()

# plt.figure(6)
# plt.imshow(vboat.img)
# plt.show()

dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
plt.figure(0)
plt.imshow(dispU2)
plt.show()

# plt.figure(1)
# plt.imshow(dispU2)
# plt.show()

plt.figure(2)
plt.imshow(vboat.vmap_processed**2)
plt.show()

dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
plt.figure(3)
plt.imshow(dispV2)
plt.show()

# plt.figure(4)
# plt.imshow(dispV2)
# plt.show()

# plt.figure(5)
# plt.imshow(compV)
# plt.show()

In [ ]:
tImg = np.copy(vboat.img)

raw_umap, raw_vmap, dt = vboat.get_uv_map(tImg)

# =========================================================================
dead_y = vboat.dead_y; dead_x = vboat.dead_x

deadzoneU = raw_umap[1:dead_y+1, :]
_, deadzoneU = cv2.threshold(deadzoneU, 80, 255,cv2.THRESH_BINARY)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# =========================================================================

threshs = [11, 10, 30, 40,40,40]
stripsPu = []
stripsU = strip_image(raw_umap, nstrips=len(threshs))

strU1 = stripsU[0]
strU2 = stripsU[1]

for i, strip in enumerate(stripsU):
#     _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_TOZERO)
    stripsPu.append(tmpStrip)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
stripsPu[0] = cv2.morphologyEx(stripsPu[0], cv2.MORPH_CLOSE, kernel)

tK = np.ones((3,2),np.uint8)
copy = stripsPu[1].copy()

stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_CLOSE, kernel)

# stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_OPEN, (2,2))
# stripsPu[1] = cv2.blur(stripsPu[1],(2,2))
tmpp = cv2.morphologyEx(stripsPu[1], cv2.MORPH_GRADIENT, kernel)
# stripsPu[1] = cv2.equalizeHist(stripsPu[1])
# stripsPu[2] = cv2.morphologyEx(stripsPu[2], cv2.MORPH_OPEN, kernel)
pplot(tmpp, 'tmpp')
stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_BLACKHAT, kernel)

# ============================ CRAP ============================
kernel = np.ones((4,4),np.uint8)

_, tmpStrip = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY)
# _, tmpStrip2 = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY_INV)
# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
dist_transform = cv2.distanceTransform(tmpStrip,cv2.DIST_L2,5)
# dist_transform2 = cv2.distanceTransform(tmpStrip2,cv2.DIST_L2,5)
# pplot(dist_transform, 'dist_transform')
# pplot(dist_transform2, 'dist_transform2')

# erode = cv2.dilate(dist_transform2,kernel,iterations=5)
erode = cv2.erode(dist_transform,kernel,iterations=1)
# stripsPu[1] = np.uint8(erode)

# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
# erode = cv2.erode(dist_transform,kernel,iterations=2)
# _, sure_fg = cv2.threshold(erode,0.1*erode.max(),255,0)

# sure_bg = cv2.dilate(sure_fg,kernel,iterations=1)
# unknown = cv2.subtract(sure_bg,sure_fg)

# # pplot(dist_transform, 'dist_transform')
# # pplot(sure_fg, 'sure_fg')
# # pplot(erode, 'erode')

# sure_fg = np.uint8(sure_fg)
# _, markers = cv2.connectedComponents(sure_fg)


# markers = markers+1
# # Now, mark the region of unknown with zero
# markers[unknown==255] = 0
# pplot(markers, 'markers')

# tmp = cv2.cvtColor(stripsPu[1],cv2.COLOR_GRAY2BGR)
# markers = cv2.watershed(tmp,markers)
# tmp[markers == -1] = [255,0,0]

# pplot(tmp, 'tmp')
# ============================ CRAP ============================


hUs,w = stripsPu[0].shape[:2]
blankStrip = np.zeros((hUs-dead_y,w),dtype=np.uint8)
deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
# try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
# except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

# print(stripsPu[0].shape, deadzone_mask.shape)
stripsPu[0] = cv2.addWeighted(stripsPu[0], 1.0, deadzone_mask, 1.0, 0)
pplot(stripsPu[0], 'stripsPu[0] After')

tmpMax = np.max(stripsPu[0])
# print(tmpMax)
dead_strip = stripsPu[0][0:14, :]
rest_strip = stripsPu[0][14:stripsPu[0].shape[0], :]

dead_strip[dead_strip < tmpMax*0.4] = 0
rest_strip[rest_strip < tmpMax*0.1] = 0
stripsPu[0] = np.concatenate((dead_strip,rest_strip), axis=0)


# pplot(stripsPu[1], 'stripsPu[1] After')



pplot(deadzone_mask, 'Deadzone Mask')
   
pUmap = np.concatenate(stripsPu, axis=0)

cnt_threshs = [55.0,50.0,80.0,40.0,40.0,40.0]
contours = []
for i, strip in enumerate(stripsPu):
    contours += vboat.find_contours(strip, cnt_threshs[i], offset=(0,hUs*i))
# contours = fCnts1 + fCnts2 + fCnts3 + fCnts4 + fCnts5 + fCnts6
print(len(contours))

disp = pUmap.copy()
for cnt in contours:
    cv2.drawContours(disp, [cnt], 0, (255,0,255), 1)
    
pplot(disp, 'Filtered Umap')

xLims, dLims, _ = vboat.extract_contour_bounds(contours)
# ==============================================================================
#                           V MAP
# ==============================================================================
threshsV = [5, 20, 40,40,40]
stripsV = strip_image(raw_vmap, nstrips=len(threshsV), horizontal_strips=False)

pplot(stripsV[0], 'stripsPv[0] Before')
# pplot(stripsV[1], 'stripsPv[1] Before')

stripsPv = []
for i, strip in enumerate(stripsV):
#     _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_TOZERO)
    stripsPv.append(tmpStrip)
    
tmpMax = np.max(stripsPv[0])
# print(tmpMax)
stripsPv[0][stripsPv[0] < np.max(stripsPv[0]*0.035)] = 0

top_strip = stripsPv[0][0:100, :]
bot_strip = stripsPv[0][100:stripsPv[0].shape[0], :]
    
top_strip[top_strip < tmpMax*0.1] = 0
bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[0] = np.concatenate((top_strip,bot_strip), axis=0)

# tmpMax = np.max(stripsPv[1])
tH = stripsPv[1].shape[0]
top_strip = stripsPv[1][0:tH/2, :]
bot_strip = stripsPv[1][tH/2:tH, :]

top_strip[top_strip < tmpMax] = 0
# bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)    
    
    
# pplot(stripsPv[0]**2, 'stripsPv[0] After')
# pplot(stripsPv[1]**2, 'stripsPv[1] After')
    
newV = np.concatenate(stripsPv, axis=1)
tmpV = cv2.cvtColor(newV, cv2.COLOR_GRAY2BGR)
mask, maskInv,mPxls, ground_wins,_ = vboat.get_vmap_mask(tmpV, maxStep = 15)
vmap = cv2.bitwise_and(newV,newV,mask=maskInv)
pplot(vmap*10, 'Vmap After')
# =========================================================================

obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, dLims, xLims)

dispObs = cv2.cvtColor(tImg, cv2.COLOR_GRAY2BGR)

for ob in obs:
    cv2.rectangle(dispObs,ob[0],ob[1],(150,0,0),1)
    
print(len(obs))
pplot(dispObs, 'Obstacles')
# pplot(strU1, 'stripsPu[0] Before')
# pplot(strU2, 'stripsPu[1] Before')

In [ ]:
pklParentDir = "/home/hyoung/data/vboats/test_keyParams"
sprImg,fact = spread_image(strU1)
sub1 = strU1 - sprImg[:42,:]
pplot(sub1,'Spread U1')

desprdImg = despread_image(sprImg,fact)
print(desprdImg.shape)
sub2 = strU1 - desprdImg[:42,:]
pplot(sub2,'Spread U2')
pplot(desprdImg,'DeSpread U2')


In [ ]:
ellipses = []
disp = np.copy(vboat.umap_processed)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for cnt in vboat.filtered_contours:
    ellipse = cv2.fitEllipse(cnt)
    ellipses.append(ellipse)
    cv2.ellipse(disp,ellipse,(0,255,255),1)
    
plt.figure(1)
plt.imshow(disp)
plt.show()

In [ ]:
img = np.copy(vboat.img)
print(vboat.obstacles[0])
win = vboat.obstacles[0]
print(win[0][0], win[1][0])

rectsTypeA = [] # x0, y0, x1, y1
rectsTypeB = [] # x, y, w, h
for win in vboat.obstacles:
    
    wid = win[1][0] - win[0][0]
    h = win[1][1] - win[0][1]
    cx = win[0][0] + wid/2
    cy = win[0][1] + h/2
    
    tmpA = [win[0][0], win[0][1], win[1][0], win[1][1]]
    tmpB = [cx, cy, wid, h]
    rectsTypeA.append(tmpA)
    rectsTypeB.append(tmpB)


# cv2.circle(img,(pxl[0],pxl[1]),2,(255,0,255),-1)

print(wid, h)
print(cx, cy)
print(rectsTypeB)
print(vboat.nObs)

In [ ]:
n = vboat.nObs
idcs = []
pick = []
boxes = np.array(rectsTypeA)
x1 = boxes[:,0]
y1 = boxes[:,1]
x2 = boxes[:,2]
y2 = boxes[:,3]
area = (x2 - x1 + 1) * (y2 - y1 + 1)
idxs = np.argsort(y2)
overlapThresh = 0.05
supressed = []
top = []
non = []
print(idxs)
while len(idxs) > 0:
    # grab the last index in the indexes list, add the index
    # value to the list of picked indexes, then initialize
    # the suppression list (i.e. indexes that will be deleted)
    # using the last index
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)
    suppress = [last]
    # loop over all indexes in the indexes list
    print(i)
    overlaps = False
    for pos in xrange(0, last):
        # grab the current index
        j = idxs[pos]

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = max(x1[i], x1[j])
        yy1 = max(y1[i], y1[j])
        xx2 = min(x2[i], x2[j])
        yy2 = min(y2[i], y2[j])

        # compute the width and height of the bounding box
        w = max(0, xx2 - xx1 + 1)
        h = max(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding box in the area list
        
        overlap = float(w * h) / area[j]

        print("Window %d Overlaps %.4f with Rectangle %d" % (j,overlap, i) )
        # if there is sufficient overlap, suppress the
        # current bounding box
        if overlap > overlapThresh:
#             suppress.append(pos)
            supressed.append(pos)
            top.append([pos,i])
            overlaps = True

    # delete all indexes from the index list that are in the
    # suppression list
    print("suppressing idxs", suppress)
#     supressed.append(suppress)
    idxs = np.delete(idxs, suppress)
    if(not overlaps):
        non.append(last)

rectangles = [] 
for a,b in top:
    rect = union(rectsTypeA[a],rectsTypeA[b])
    rectangles.append(rect)
    

print(top)
print(non)
print(supressed)
mask = np.isin(non, supressed, invert=True)
remain = np.where(mask)
print(remain)
rectangles.append(rectsTypeA[remain[0][0]])

print(rectangles)
for i in range(n):
    has_overlap = False
    for j in range(n):
        if(i is not j):
            iou = test_intersection(rectsTypeA[i],rectsTypeA[j])
            if(iou >= 0.05):
                has_overlap = True
                idcs.append([i, j])
                print("IoU for Rect %d and Rect %d = %.3f" % (i, j, iou))
    if(not has_overlap):
        print("Rect %d has no overlapping segments" % i)

In [ ]:
disp = np.copy(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for x1,y1,x2,y2 in boxes[pick]:
    cv2.rectangle(disp,(x1,y1),(x2,y2),(0,255,255), 1)
    
plt.figure(10)
plt.imshow(disp)
plt.show()

In [ ]:
disp = np.zeros_like(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
masks = []
for win in vboat.obstacles:
    mask = np.copy(disp)
    cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks.append(mask)
    

tmp = cv2.bitwise_and(vboat.img,vboat.img, mask=masks[0])

tmpMean = np.mean(tmp)
print(tmpMean/0.001)
plt.figure(2)
plt.imshow(tmp)
plt.show()

np.amin()

In [ ]:
newStrips = []
raw_umap = vboat.umap_raw; dead_y = 3
umap = np.copy(raw_umap)

cv2.rectangle(umap,(0,0),(umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(umap,(0,umap.shape[0]-dead_y),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

threshsU = [0.15, 0.15, 0.075, 0.065,0.05,0.025]
n = len(threshsU)

stripsU = strip_image(umap, nstrips=n)
strips = strip_image(stripsU[0], nstrips=n)
strips2 = strip_image(stripsU[1], nstrips=n)

# print(np.max(umap))
absmax = np.max(umap)
dmax = absmax
stripmax = np.max(stripsU[0])
# print(dmax,stripmax)
baseThreshs = []
absMaxs=[absmax,stripmax]
absRatios = []
relMaxs = []
avgVals = []
threshs = []
for i, strip in enumerate(strips):
    dratio = dmax/255.0
    tmpMax = np.max(strip)
    relRatio = (dmax-tmpMax)/float(dmax)
    avgd = strip[np.nonzero(strip)].mean()
    if np.isnan(avgd):
        avgd = np.nan_to_num(avgd)
    avgRatio = math.fabs(avgd)/float(dmax)
    
    baseThreshs.append(threshsU[i])
    absRatios.append(dratio)
    relMaxs.append(tmpMax)
    avgVals.append(int(avgd))
    
    if(dratio <= 0.35):
        tmpGain = 10.0
        threshGain = 0.6
        idxPerc = (1+i)/float(n)
        if idxPerc > 0.5: threshGain = threshGain + 0.25
#         if i == 0: threshAdder = 0.35
        if i <= 1: threshAdder = math.fabs(relRatio-dratio)
        else: threshAdder = 0
    else:
#         tmpGain = 1.0
        tmpGain = dratio
#         threshGain = 1 - relRatio
        threshGain = 1-avgRatio
        if i == 0: threshAdder = avgRatio #0.15
        else: threshAdder = 0
    
    
    if relRatio == 0: ratioGain = 0
    else: ratioGain = avgRatio#(relRatio)*
        
    if i == 0: gain = threshsU[i]*relRatio+ ratioGain*0# + threshAdder
    else:
        gain = threshsU[i]*threshGain*dratio*tmpGain + threshAdder
        gain = gain * 1.5
    
#     gain = (avgRatio) * gain * 10
#     gain = 1-gain
#     gain = math.fabs(gain)
#     gain = threshsU[i]*threshGain*(math.fabs(relRatio-dratio))*tmpGain + threshAdder

#     testGain = threshsU[i]*threshGain*relRatio*tmpGain + threshAdder
    testGain = threshsU[i]*threshGain*dratio*tmpGain + threshAdder
    testGain = testGain*1.5
    if relRatio == 0: tratioGain = 0
    else: tratioGain = (1-relRatio)
    testGain2 = threshsU[i]+(tratioGain)
    testThresh = int(testGain * dmax)
    testThresh2 = int(testGain2 * dmax)
    tmpThresh = int(gain * dmax)
    threshs.append(tmpThresh)
    
    if relRatio == 0: t2ratioGain = dratio
    else: t2ratioGain = relRatio

#     print(avgd, avgRatio, ratioGain/dratio)
#     print("max, relative max, totalPerc, relativePerc: %d, %d, %.3f, %.3f" % (dmax,tmpMax,dratio,relRatio))
#     print("total gain, relative gain: %.3f, %.3f" % (testGain,testGain2))
#     print("thresh w/ total max, relative max: %d, %d" % (testThresh,testThresh2))
#     print("dratio, relRatio, pre_gain, gain, adder, thresh: %.3f, %.3f, %.3f, %.3f, %.3f, %d" %
#           (dratio,relRatio, threshGain,gain,threshAdder,tmpThresh))
    
#     _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_BINARY)
    _,tmpStrip = cv2.threshold(strip, tmpThresh,255,cv2.THRESH_TOZERO)
    
    newStrips.append(tmpStrip)
    
newUmap = np.concatenate(newStrips, axis=0)

ksize = (15,2)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,ksize)
pUmap = cv2.morphologyEx(newUmap, cv2.MORPH_CLOSE, kernel)

pplots(newStrips,"Thresholded Umap 0", flag_resize=True,scale=(5,1))
pplots(strips,"Raw Umap 0", flag_resize=True,scale=(5,1))
# pplots(strips2,"Raw Umap 1", flag_resize=True,scale=(5,1))

contours = vboat.find_contours(pUmap, 50.0)
disp = cv2.applyColorMap(pUmap,cv2.COLORMAP_PARULA)
disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
pplot(disp,"Umap")
pplot(overlay,'Overlay')

plist("absMaxs: ", absMaxs)
plist("relMaxs: ", relMaxs)
plist("threshs: ", threshs)
plist("avgVals: ", avgVals)

In [ ]:
phase = np.linspace(-np.pi, np.pi, num=10)

In [ ]:
print np.rad2deg(phase)
print np.rad2deg(phase+np.pi)

In [ ]:
# unphase = np.unwrap(phase)
unphase = ( phase+ np.pi) % (2 * np.pi )
print np.rad2deg(unphase)

In [ ]:
angles_2pi = np.mod(phase, 2*np.pi)
print np.rad2deg(angles_2pi)

In [ ]:
angs = [x if x > 0 else np.mod(x+np.pi, 2*np.pi) for x in phase]
angs = []
for x in phase:
    ang = np.arcsin(np.sin(x))
    if np.cos(x) < 0: ang = np.pi - ang
    elif ang < 0: ang+=2*np.pi
    angs.append(ang)

print np.rad2deg(angs)

KeyboardInterrupt: 

206


In [57]:
found_depths, depths = grab_dir_images("/home/hunter/data/vboats/door_finding_tests/test_0/depth")
found_rgbs,rgbs= grab_dir_images("/home/hunter/data/vboats/door_finding_tests/test_0/rgb")


# imgPath = "/home/hunter/data/vboats/door_finding_tests/test_0/depth/depth_frame_6.png"
imgPath = "/home/hunter/data/vboats/door_finding_tests/test_0/depth/depth_frame_5.png"
img = cv2.imread(imgPath,cv2.IMREAD_GRAYSCALE)

idx = 0
# img = depths[idx]
# pplot(rgbs[idx],"rgb")
pplot(img,"depth")
umap,vmap,_ = vboat.get_uv_map(img)
pplot(umap,"umap")
_ =cv2.rectangle(umap,(0,umap.shape[0]-10),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

tmpMax = np.max(umap)
tmpThresh = int(0.02 * tmpMax)
# _, filt = cv2.threshold(umap, tmpThresh, 255,cv2.THRESH_TOZERO)


kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(25,1))
# filt = cv2.morphologyEx(filt, cv2.MORPH_CLOSE, kernel)
filt = cv2.morphologyEx(umap, cv2.MORPH_OPEN, kernel)

_, filt = cv2.threshold(filt, tmpThresh, 255,cv2.THRESH_TOZERO)
pplot(filt,"filt")

contours,raw_cnts = vboat.find_contours(filt,threshold=50.0,debug=True, max_thresh=-1,)
# xBounds, disparityBounds, _ = vboat.extract_contour_bounds(raw_cnts)
xBounds, disparityBounds, _ = vboat.extract_contour_bounds(contours)
print(disparityBounds)
print(xBounds)

maxDbs = 0
bestIdx = 0
for i, dbs in enumerate(disparityBounds):
    avgD = np.mean(dbs)
    if avgD >= maxDbs:
        maxDbs = avgD
        bestIdx = i
        print("found better candidate", maxDbs,bestIdx)
print("best candidates xbouds = ",xBounds[bestIdx])

dist,x,y,z = vboat.calculate_distance(umap,xBounds[bestIdx],disparityBounds[bestIdx],[1,2],verbose=True)

tmpU = np.copy(umap)
# tmpU = cv2.normalize(tmpU,None,8191.0)
s = cv2.applyColorMap(tmpU,cv2.COLORMAP_PARULA)
cv2.drawContours(s, [contours[bestIdx]], 0, (0,0,255), 2)
disp = cv2.cvtColor(s,cv2.COLOR_BGR2RGB)
pplot(disp,"found entry")

('Raw Contour Perimeters:', array([   7.65685415,   52.        ,   54.        ,   59.65685415,
         80.        ,  132.82842708,  246.        ,  358.        ,
       1217.65685415, 1275.94112492]))
('Filtered Contour Perimeters:', array([  52.        ,   54.        ,   59.65685415,   80.        ,
        132.82842708,  246.        ,  358.        , 1217.65685415,
       1275.94112492]))
[[26, 27], [22, 24], [9, 18], [9, 12], [2, 4], [0, 1], [0, 1], [0, 1], [0, 1]]
[[316, 344], [305, 372], [7, 640], [596, 626], [30, 639], [460, 640], [294, 418], [135, 162], [0, 41]]
('found better candidate', 26.5, 0)
('best candidates xbouds = ', [316, 344])
X, Y, Z: 0.120, -3.272, 6.555


In [ ]:
from scipy import optimize

def piecewise_linear(x, x0, y0, k1, k2):
    return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])

p , e = optimize.curve_fit(piecewise_linear, good[:,0], good[:,1])
xd = np.linspace(0, 15, 100)


yd = piecewise_linear(xd, *p)
print(yd)

plt.figure(4)
plt.plot(xd, piecewise_linear(xd, *p))
plt.show()

In [ ]:
model = piecewise(good[:,0], good[:,1])
print(model)

In [ ]:
# std
from collections import namedtuple
import heapq

# 3p
import numpy as np


## Function to learn and plot piecewise regressions.


def piecewise(t, v, min_stop_frac=0.03):
    """ Fits a piecewise (aka "segmented") regression.
    Params:
        t (listlike of ints or floats): independent/predictor variable values
        v (listlike of ints or floats): dependent/outcome variable values
        min_stop_frac (float between 0 and 1): the fraction of total error that
            a merge must account for to be considered big enough to stop merging;
            the default is usually adequate, but this may be increased to make
            merging more aggressive (leading to fewer segments in the result)
    Returns:
        A FittedModel object that can be used for interpolation and extrapolation.
    """
    # Validate the inputs, and force t and v to be np.arrays sorted in
    # ascending t order.
    t, v = _preprocess(t, v)

    # Initialize the segments.
    init_segments, merges = _get_initial_segments_and_merges(t, v)
    seg_tracker = SegmentTracker(init_segments)

    # Use a min heap to track potential merges. At the top of the heap
    # will be the next best merge (smallest increase in error).
    heapq.heapify(merges)

    # Greedily make the next best merge until we've merged everything together into
    # one segment.
    cum_cost, biggest_cost_increase = 0.0, 0.0

    # list of merges we need to undo to return to last best configuration
    merges_since_best = []

    while len(seg_tracker) > 1:
        # Identify the next merge to be executed.
        next_merge = _get_next_merge(merges, seg_tracker)

        # If the next merge increases the error by a larger amount than any
        # merge so far, remember the current state (which might end up being the
        # "best"). To prevent stopping too early (for example, in cases where
        # there should only be one segment), use min_stop_frac to keep on
        # remembering the current state as the best state if no single
        # merge has accounted for a significant part of the total error.
        cum_cost += next_merge.cost
        cost_increase = next_merge.cost - biggest_cost_increase
        biggest_cost_increase = max(biggest_cost_increase, cost_increase)
        if biggest_cost_increase < min_stop_frac*cum_cost or \
                cost_increase == biggest_cost_increase:
            merges_since_best = [next_merge]
        else:
            merges_since_best.append(next_merge)

        # Execute the next merge.
        # Update segments, replacing the two old ones with the one new one.
        seg_tracker.apply_merge(next_merge)
        
        # Add new potential merges.
        neighbors = seg_tracker.get_neighbors(next_merge.new_seg)
        for neighbor in neighbors:
            left_seg, right_seg = sorted([next_merge.new_seg, neighbor])
            heapq.heappush(merges, _make_merge(t, v, left_seg, right_seg))

    if biggest_cost_increase < min_stop_frac*cum_cost:
        # This path is needed for the case where there is only one segment, because
        # merges_since_best isn't updated after merging in the loop above.
        merges_since_best = []
    
    for merge in reversed(merges_since_best):
        seg_tracker.unapply_merge(merge)

    fitted_segments = [
        FittedSegment(t[seg.start_index], t[min(seg.end_index, len(t)-1)], seg.coeffs)
        for seg in seg_tracker.segments
    ]
    return FittedModel(fitted_segments)


## Data structures used for representing the fitted model returned by `piecewise()`.


class FittedSegment(namedtuple('FittedSegment',
    [
        'start_t',  # (float) first t value to which this segment applies
        'end_t',    # (float) first t value to which this segment no longer applies
        'coeffs'    # (tuple of floats) regression coefficients
    ]
)):
    def predict(self, t_new, out=None, where=True):
        return _predict(self.coeffs, t_new, out=out, where=where)


class FittedModel(object):
    """ Completely defines the result of a piecewise regression.
    The `segments` attribute contains a list of FittedSegments.
    """

    def __init__(self, fitted_segments):
        self.segments = fitted_segments
        self._starts = [fs.start_t for fs in fitted_segments]

    def __repr__(self):
        return 'FittedModel with segments:\n' + '\n'.join(
            ['* ' + seg.__repr__() for seg in self.segments]
        )

    def predict(self, t_new):
        """ Use the segments in this model to predict the v value for new t values.
        Params:
            t_new (scalar or array like): t values for which predictions should be made
        Returns:
            scalar or array like of predictions
        """
        if len(self.segments) == 1:
            return self.segments[0].predict(t_new)

        t_array = np.asanyarray(t_new)
        seg_index = np.digitize(t_array, [s.end_t for s in self.segments[:-1]])
        if seg_index.shape == (): # t_new is a scalar or 0-dimensional array
            return self.segments[seg_index].predict(t_new)
        else:
            v_hats = np.empty_like(t_array, dtype=np.double)
            for i, segment in enumerate(self.segments):
                segment.predict(t_array, out=v_hats, where=seg_index == i)
            return v_hats


## Data structures used during the fitting of the regression in `piecewise()`.


# Segment represents a time range and a linear regression fit through it.
Segment = namedtuple('Segment',
    [
        'start_index',  # (int) zero-based index of start time
        'end_index',    # (int) zero-based index of non-inclusive end time
        'coeffs',       # (tuple of floats) regression coefficients
        'error',        # (float) the total error in the segment
        'cov_data',     # (np.array of floats) incremental covariance data
    ]
)


# Merge represents a potential merge of two neighboring segments.
Merge = namedtuple('Merge',
    [
        'cost',       # (float) increase in sum of squared error that would result from executing this merge
        'left_seg',   # (Segment)
        'right_seg',  # (Segment)
        'new_seg'     # (Segment) the Segment that would result from merging combining left_seg and right_seg
    ]
)


class SegmentTracker(object):
    """ Utility class for tracking the state of the piecewise regression (i.e.,
    what are the current segments based on the set of merges that have been
    executed so far).
    """

    def __init__(self, segments):
        # Assume segments are sorted
        starts = np.fromiter((s.start_index for s in segments), np.intp, count=len(segments))

        # One position for each original index.
        # About 50% of this space is wasted, but this enables O(1) lookup and
        # replacement by start_index
        self._segments = np.empty(segments[-1].end_index, dtype=object)
        self._segments[starts] = segments

        # Valid mask.  As segments are merged, this mask is updated
        self._valid = np.not_equal(self._segments, None)

        # Previous neighbor lookup
        self._prev = np.zeros_like(self._segments, dtype=np.intp)
        self._prev[starts[1:]] = starts[:-1]

        # Cached length.  Without this, we would need to count _valid every len()
        self._len = len(segments)

    def __len__(self):
        return self._len

    def contains(self, segment):
        """ Returns True if segment is currently valid; False otherwise. """
        # segment at start_index has not been merged away and is still the same
        return self._valid[segment.start_index] \
            and self._segments[segment.start_index] is segment
    
    def get_prev(self, segment):
        """ Returns the left neighbor of segment; None if it is the first. """
        if segment.start_index > 0:
            return self._segments[self._prev[segment.start_index]]
        else:
            return None
    
    def get_next(self, segment):
        """ Returns the right neighbor of segment; None if it is the last. """
        if segment.end_index < len(self._segments):
            return self._segments[segment.end_index]
        else:
            return None

    def get_neighbors(self, segment):
        """ Returns a list of Segments, containing the 0, 1, or 2 segments
        adjacent to the given Segment.
        """
        return (
            s for s in (self.get_prev(segment), self.get_next(segment))
            if s is not None
        )

    def apply_merge(self, merge):
        """ Insert a new segment and remove the two existing segments
        from which it was created.
        """
        right_seg, new_seg = merge.right_seg, merge.new_seg
        self._valid[right_seg.start_index] = False
        _next = self.get_next(right_seg)
        if _next:
            self._prev[_next.start_index] = new_seg.start_index
        self._segments[new_seg.start_index] = new_seg
        self._len -= 1
    
    def unapply_merge(self, merge):
        """ Remove a segment and reinsert the two segments
        from which it was created.
        """
        right_seg, left_seg = merge.right_seg, merge.left_seg
        self._valid[right_seg.start_index] = True
        _next = self.get_next(right_seg)
        if _next:
            self._prev[_next.start_index] = right_seg.start_index
        self._segments[left_seg.start_index] = left_seg
        self._len += 1

    @property
    def segments(self):
        return self._segments[self._valid]


## Helper functions for doing piecewise regression.


def _preprocess(t, v):
    """ Raises an exception if any of the inputs are not valid.
    Otherwise, returns a list of Points, ordered by t.
    """
    # Validate the inputs.
    if len(t) != len(v):
        raise ValueError('`t` and `v` must have the same length.')
    t_arr, v_arr = np.asanyarray(t, dtype=np.double), np.asanyarray(v, dtype=np.double)
    if not np.all(np.isfinite(t)):
        raise ValueError('All values in `t` must be finite.')
    finite_mask = np.isfinite(v_arr)
    if np.sum(finite_mask) < 2:
        raise ValueError('`v` must have at least 2 finite values.')
    t_arr, v_arr = t_arr[finite_mask], v_arr[finite_mask]

    # Order both arrays by t-values.
    sort_order = np.argsort(t_arr)
    t_arr, v_arr = t_arr[sort_order], v_arr[sort_order]

    return t_arr, v_arr


def _get_initial_segments_and_merges(t, v):
    """ Returns a 2-tuple with the lists of initial segments and initial merges.
    Each Segment is of length 1, 2, or 3. They are created by using even-indexed
    points as seeds and attaching odd-indexed points to the neighboring seed with
    the closer v value.
    This initialization procedure exists to decrease the odds of bad initial
    merges. If initial segments were each a single point, then merging any two
    neighboring points would be equally attractive to our algorithm, because the
    squared error of a line fit through any pair of points is zero. However,
    in the case that the data looks like [1, 1, 1, 1, 10, 10, 10, 10], we would
    prefer to avoid the 1 and neighboring 10 from starting out in the same
    segment. This initialization does this by doing initial merges based on
    absolute difference rather than regression error. Unfortunately, there can
    still be suboptimal initializations, as in this case, where the two 1s will
    be initialized in the same segment: [19, 10, 1, 1, -8, -17]
    """

    # creates segments from an array of start, end indices
    def _build_segments(ranges):
        # number of point in range
        n = np.diff(ranges, axis=1).reshape(-1)
        
        # expand ranges in rows, using masked arrays to deal with uneven lengths.
        # indices like these:
        #   [[0, 1], [1, 4], [4, 6]]
        # yield something like this:
        #   [
        #       [v0, --, --],
        #       [v1, v2, v3],
        #       [v4, v5, --],
        #   ]
        max_n = np.max(n)
        indices = np.ma.array(ranges[:,:1] + np.arange(max_n).reshape(1, -1))
        for i in range(1, max_n):
            indices[n == i, i:] = np.ma.masked
        
        segment_t = np.ma.take(t, indices)
        segment_v = np.ma.take(v, indices)
    
        # sum(t), sum(v), unmasked
        st = np.ma.getdata(np.ma.sum(segment_t, axis=1))
        sv = np.ma.getdata(np.ma.sum(segment_v, axis=1))

        # mean(t), mean(v)
        mu_t = (st / n).reshape(-1, 1)
        mu_v = (sv / n).reshape(-1, 1)

        # distance from means
        dt = segment_t - mu_t
        dv = segment_v - mu_v

        # var(t), var(v) and cov(t, v), before division by n, unmasked
        ct = np.ma.getdata(np.ma.sum(dt ** 2, axis=1))
        cv = np.ma.getdata(np.ma.sum(dv ** 2, axis=1))
        ctv = np.ma.getdata(np.ma.sum(dt * dv, axis=1))
        
        # slope and intercept
        # for single point segments (ct == 0), assume slope = 0, intercept = mean(v)
        nonzero_ct = ct > 0
        slope = np.where(nonzero_ct, ctv, 0.0)
        slope = np.divide(slope, ct, out=slope, where=nonzero_ct).reshape(-1, 1)
        intercept = mu_v - slope * mu_t

        # sum of squared errors
        # if n < 3: error = 0
        # elif ct == 0: error = cv
        # else: error = cv - ctv ** 2 / ct

        nonzero_error = n >= 3
        nonzero_ct &= nonzero_error
        error = np.where(nonzero_ct, ctv, 0.0) # 0, 0, ctv
        np.square(error, out=error, where=nonzero_ct) # 0, 0, ctv ** 2
        np.divide(error, ct, out=error, where=nonzero_ct) # 0, 0, ctv ** 2 / ct
        np.subtract(cv, error, out=error, where=nonzero_error) # 0, cv, cv - ctv ** 2 / ct


        return [
            Segment(
                ranges[i, 0], ranges[i, 1], (intercept[i, 0], slope[i, 0]), error[i],
                cov_data
            )
            for i, cov_data in enumerate(np.c_[n, st, sv, ct, cv, ctv])
        ]
    
    # If there are multiple values at the same t, average them and treat them
    # like a single point during initialization. This ensures that all the
    # points with the same t are assigned to the same linear segment.
    unique_t = np.unique(t, return_index=True)[1]
    even_n = len(unique_t) % 2 == 0
    index_ranges = np.c_[unique_t, np.r_[unique_t[1:], len(t)]]
    
    # unique t is pretty common, optimize for that
    averages = v[index_ranges[:,0]]
    long_ranges = np.diff(index_ranges, axis=1).reshape(-1) > 1
    if long_ranges.any():
        averages[long_ranges] = np.fromiter(
            (v[idx[0]:idx[1]].mean() for idx in index_ranges[long_ranges]),
            np.double, count=long_ranges.sum()
        )

    # Pair every other t with the t on its left or on its right, based on which
    # is closer.
    pair_left = np.less(*np.abs(
        np.ediff1d(averages, to_end=np.inf if even_n else None)
    ).reshape(-1, 2).T)
    np.copyto(
        index_ranges[:-1:2, 1],
        index_ranges[1::2, 1],
        where=pair_left
    )
    np.copyto(
        index_ranges[2::2, 0],
        index_ranges[1:-1:2, 0],
        where=~pair_left[:-1 if even_n else None]
    )

    # initial segment ranges are at even indices
    segment_ranges = index_ranges[::2]
    segments = _build_segments(segment_ranges)

    # merge every consecutive segment
    merge_ranges = np.c_[segment_ranges[:-1,0], segment_ranges[1:,1]]
    merge_segments = _build_segments(merge_ranges)
    
    merges = [
        Merge(
            new_seg.error - segments[i].error - segments[i + 1].error,
            segments[i], segments[i + 1], new_seg
        )
        for i, new_seg in enumerate(merge_segments)
    ]

    return segments, merges


def _get_next_merge(merges, segment_tracker):
    """ Returns the valid Merge that has the lowest cost.
    Params:
        merges: a heapified list of Merges
        segment_tracker: a SegmentTracker with the currently valid segments;
            any Merge referencing a Segment not in the tracker is no longer valid
    """
    while True:
        next_merge = heapq.heappop(merges)
        if (segment_tracker.contains(next_merge.left_seg) and
                segment_tracker.contains(next_merge.right_seg)):
            return next_merge


def _make_segment(t, v, left_seg, right_seg):
    """ Returns a Segment that is the merge of left_seg and right_seg,
    starting at left_seg.start_index and ending at the non-inclusive
    right_seg.end_index.
    """
    start_index = left_seg.start_index
    end_index = right_seg.end_index
    cov_data = _merge_cov_data(left_seg.cov_data, right_seg.cov_data)
    coeffs, error = _fit_line(t, v, start_index, end_index, cov_data)
    return Segment(start_index, end_index, coeffs, error, cov_data)


def _make_merge(t, v, left_seg, right_seg):
    """ Returns a Merge combining the left_seg and right_seg Segments.
    """
    new_seg = _make_segment(t, v, left_seg, right_seg)
    cost = new_seg.error - left_seg.error - right_seg.error
    return Merge(cost, left_seg, right_seg, new_seg)


def _merge_cov_data(d1, d2):
    """ Merge covariance data from two segments into a new one.
    See also:
        https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Parallel_algorithm
        https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Online
    """
    d3 = d1 + d2
    n1 = d1[0]
    n2 = d2[0]
    n12 = n1 * n2
    n3 = d3[0]
    deltat = (d1[1] * n2 - d2[1] * n1) / n12
    deltav = (d1[2] * n2 - d2[2] * n1) / n12
    d3[3] += deltat ** 2 * n12 / n3
    d3[4] += deltav ** 2 * n12 / n3
    d3[5] += deltat * deltav * n12 / n3
    return d3

def _fit_line(t, v, start_index, end_index, cov_data):
    """ Fits and OLS regression for the set of t and v values in the given index
    range. Returns (coefficients of line, sum of squared error).
    """

    # based on scipy.stats.linregress
    mu_t, mu_v = cov_data[1:3] / cov_data[0]
    ct, cv, ctv = cov_data[3:]
    if ct != 0:
        slope = ctv / ct
        intercept = mu_v - slope * mu_t
        error = cv - ctv ** 2 / ct
    else:
        slope, intercept, error = 0.0, mu_v, cv

    return ((intercept, slope), error)


def _predict(coeffs, t, out=None, where=True):
    """ Given OLS coefficients, predict the corresponding v values for the given
    t values.
    """
    # if out is None, numpy allocates an empty one
    out = np.multiply(t, coeffs[1], out=out, where=where)
    if np.isscalar(out):
        # t was either a scalar or a 0-dimensional array
        # returning a scalar is consistent with numpy arithmetic operations
        return out + coeffs[0]
    else:
        np.add(out, coeffs[0], out=out, where=where)
        return out


In [ ]:
def piecewise_plot(t, v, min_stop_frac=0.03, model=None):
    """ Fits a piecewise (aka "segmented") regression and creates a scatter plot
    of the data overlaid with the regression segments.
    Params:
        t (listlike of ints or floats): independent/predictor variable values
        v (listlike of ints or floats): dependent/outcome variable values
        min_stop_frac (float between 0 and 1): the fraction of total error that
            a merge must account for to be considered "too big" to keep merging;
            the default is usually adequate, but this may be increased to make
            merging more aggressive (leading to fewer segments in the result)
        model (FittedModel): a previously fit model, if available
    Returns:
        None.
    """
    # 3p
    # delay importing until first use (enables export in __init__.py)
    import matplotlib.pyplot as plt
    
    model = model or piecewise(t, v, min_stop_frac)
    print('Num segments: %s' % len(model.segments))
    plt.plot(t, v, '.', alpha=0.6)
    for seg in model.segments:
        t_new = [seg.start_t, seg.end_t]
        v_hat = [seg.predict(t) for t in t_new]
        plt.plot(t_new, v_hat, 'k-')
    plt.show()

# alias for backward compatibility
plot_data_with_regression = piecewise_plot


In [ ]:
# segs = [2:5]
ys = []
xs = []
tmpSeg = model.segments[2]
t_new = [tmpSeg.start_t, tmpSeg.end_t]
for t in t_new:
    try:
        test = float(tmpSeg.predict(t))
        if np.isinf(test) or np.isnan(test):
            continue
        elif(np.nan_to_num(test) > 10000):
            continue
        elif(np.nan_to_num(test) < -10000):
            continue
        else:
#             print(np.nan_to_num(test))
            xs.append(float(t))
            ys.append(float(np.nan_to_num(test)))
    except: pass
xs = np.round(xs,5)
ys = np.round(ys,5)

plt.figure(5)
# plt.cla()
# seg = model.segments[4]
# t_new = [seg.start_t, seg.end_t]
# v_hat = [seg.predict(t) for t in t_new]
plt.plot(list(ys), list(xs))
plt.axis('equal')
plt.show()

In [ ]:
plt.cla()

In [9]:
import rospy
from math import e, pi, isinf
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image

from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist, TwistStamped, PoseStamped
from tf.transformations import euler_from_quaternion, quaternion_from_euler

bridge = CvBridge()

In [5]:
scans = []
imgs = []
scan_lims = []
max_angle = 0.0
dTheta = 1.0

def scanCallback(msg):
    global scans,scan_lims,max_angle,dTheta
    scans.append(msg.ranges)
    scan_lims.append([msg.range_min,msg.range_max])
    max_angle = msg.angle_max
    dTheta = msg.angle_increment
    
    
def imgCallback(msg):
    global bridge
    try:
        cv_image = bridge.imgmsg_to_cv2(msg, "8UC1")
#         cv_image = np.float32(cv_image)
#         tmp = cv_image/65535
#         depth = np.uint8(tmp*255)
        imgs.append(np.copy(cv_image))
    except CvBridgeError as e:
        print(e)

In [6]:
rospy.init_node('something_node')
r = rospy.Rate(10)
_sub = rospy.Subscriber("atx0/scan", LaserScan, scanCallback, queue_size=10)
# image_sub = rospy.Subscriber("atx0/d415/depth/image_raw", Image, imgCallback)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [7]:
finish = False

while not rospy.is_shutdown():
    global finish
    r.sleep()
    if finish:
        break

KeyboardInterrupt: 

In [11]:
quats = (-0.00874631064271, -0.0123980916094,-0.858510447674,-0.512571556574)
euler = euler_from_quaternion(quats)
print(euler[2])

angs =[]
angs = np.deg2rad(np.linspace(0,360,360))+euler[2]
angs = ((angs+ np.pi) % (2 * np.pi )-np.pi)
# offset = np.rad2deg((euler[2]+ np.pi) % (2 * np.pi )-np.pi)
distHist = []
ptsHist = []
fdistHist = []
fptsHist = []

lim = np.deg2rad(135)
for scan in scans:
    pts = []
    readings = []
    filtered = []
    filtered_readings = []
    for i,rng in enumerate(scan):
        if not math.isinf(rng):
            tmp = np.nan_to_num(rng)
            x = tmp*math.cos(angs[i])
            y = tmp*math.sin(angs[i])
            pts.append([x,y])
            readings.append([rng,angs[i]])
            if 0.0 <= angs[i] <= lim:
                filtered.append([x,y])
                filtered_readings.append([rng,angs[i]])
                print("Reading of %f at %f degrees." % (rng,np.rad2deg(angs[i])))
    ptsHist.append(np.array(pts))
    distHist.append(np.array(readings))
    fdistHist.append(np.array(filtered_readings))
    fptsHist.append(np.array(filtered))
    
distHist = np.array(distHist)
ptsHist = np.array(ptsHist)
fdistHist = np.array(fdistHist)
fptsHist = np.array(fptsHist)

print fdistHist


finish = True
print(len(scans))
print(len(scans[0]),float(dTheta))
# print angs

2.06506623427
Reading of 2.729000 at 119.322365 degrees.
Reading of 2.779000 at 120.325151 degrees.
Reading of 2.777000 at 121.327936 degrees.
Reading of 2.704000 at 122.330722 degrees.
Reading of 2.654000 at 123.333507 degrees.
Reading of 2.591000 at 124.336293 degrees.
Reading of 2.522000 at 125.339078 degrees.
Reading of 2.460000 at 126.341864 degrees.
Reading of 2.405000 at 127.344649 degrees.
Reading of 2.367000 at 129.350220 degrees.
Reading of 2.309000 at 130.353006 degrees.
Reading of 2.258000 at 131.355791 degrees.
Reading of 2.224000 at 132.358577 degrees.
Reading of 2.192000 at 133.361362 degrees.
Reading of 2.156000 at 134.364148 degrees.
Reading of 0.346000 at 0.993674 degrees.
Reading of 0.334000 at 1.996460 degrees.
Reading of 0.325000 at 2.999245 degrees.
Reading of 0.319000 at 4.002031 degrees.
Reading of 0.315000 at 5.004816 degrees.
Reading of 0.311000 at 6.007602 degrees.
Reading of 0.308000 at 7.010387 degrees.
Reading of 0.306000 at 8.013173 degrees.
Reading of 0.

Reading of 2.317000 at 53.138521 degrees.
Reading of 2.305000 at 54.141307 degrees.
Reading of 2.283000 at 55.144092 degrees.
Reading of 2.261000 at 56.146878 degrees.
Reading of 2.262000 at 57.149663 degrees.
Reading of 2.292000 at 58.152449 degrees.
Reading of 2.413000 at 59.155234 degrees.
Reading of 10.673000 at 66.174733 degrees.
Reading of 2.579000 at 75.199802 degrees.
Reading of 2.530000 at 77.205374 degrees.
Reading of 2.475000 at 78.208159 degrees.
Reading of 2.399000 at 79.210945 degrees.
Reading of 2.336000 at 80.213730 degrees.
Reading of 2.279000 at 81.216516 degrees.
Reading of 2.122000 at 83.222087 degrees.
Reading of 2.126000 at 84.224872 degrees.
Reading of 2.130000 at 85.227658 degrees.
Reading of 2.121000 at 86.230443 degrees.
Reading of 2.136000 at 87.233229 degrees.
Reading of 2.137000 at 88.236014 degrees.
Reading of 2.048000 at 89.238800 degrees.
Reading of 2.050000 at 90.241585 degrees.
Reading of 2.069000 at 91.244371 degrees.
Reading of 2.075000 at 93.249942 

Reading of 1.516000 at 104.280582 degrees.
Reading of 1.513000 at 105.283368 degrees.
Reading of 1.510000 at 106.286153 degrees.
Reading of 1.511000 at 107.288939 degrees.
Reading of 1.683000 at 108.291724 degrees.
Reading of 1.882000 at 133.361362 degrees.
Reading of 1.833000 at 134.364148 degrees.
Reading of 0.267000 at 0.993674 degrees.
Reading of 0.268000 at 1.996460 degrees.
Reading of 0.272000 at 4.002031 degrees.
Reading of 0.278000 at 5.004816 degrees.
Reading of 1.835000 at 13.027101 degrees.
Reading of 1.792000 at 14.029886 degrees.
Reading of 1.739000 at 15.032672 degrees.
Reading of 1.681000 at 16.035457 degrees.
Reading of 1.635000 at 17.038243 degrees.
Reading of 1.575000 at 18.041028 degrees.
Reading of 1.516000 at 19.043814 degrees.
Reading of 1.522000 at 20.046599 degrees.
Reading of 1.361000 at 22.052170 degrees.
Reading of 1.356000 at 23.054956 degrees.
Reading of 1.352000 at 24.057741 degrees.
Reading of 1.357000 at 25.060527 degrees.
Reading of 1.361000 at 27.06609

Reading of 0.879000 at 10.018744 degrees.
Reading of 0.891000 at 11.021529 degrees.
Reading of 0.898000 at 12.024315 degrees.
Reading of 0.907000 at 13.027101 degrees.
Reading of 0.917000 at 14.029886 degrees.
Reading of 0.929000 at 15.032672 degrees.
Reading of 0.938000 at 17.038243 degrees.
Reading of 0.952000 at 18.041028 degrees.
Reading of 0.962000 at 19.043814 degrees.
Reading of 0.979000 at 20.046599 degrees.
Reading of 0.991000 at 21.049385 degrees.
Reading of 1.008000 at 22.052170 degrees.
Reading of 1.023000 at 23.054956 degrees.
Reading of 1.042000 at 24.057741 degrees.
Reading of 1.063000 at 25.060527 degrees.
Reading of 1.073000 at 26.063312 degrees.
Reading of 1.094000 at 27.066098 degrees.
Reading of 1.117000 at 29.071669 degrees.
Reading of 1.138000 at 30.074454 degrees.
Reading of 1.162000 at 31.077240 degrees.
Reading of 1.187000 at 32.080025 degrees.
Reading of 1.210000 at 33.082811 degrees.
Reading of 1.243000 at 34.085596 degrees.
Reading of 1.265000 at 35.088382 d

Reading of 1.521000 at 68.180304 degrees.
Reading of 1.516000 at 69.183089 degrees.
Reading of 1.512000 at 70.185875 degrees.
Reading of 1.503000 at 71.188660 degrees.
Reading of 1.501000 at 72.191446 degrees.
Reading of 1.498000 at 73.194231 degrees.
Reading of 1.491000 at 74.197017 degrees.
Reading of 1.490000 at 75.199802 degrees.
Reading of 1.490000 at 76.202588 degrees.
Reading of 1.488000 at 77.205374 degrees.
Reading of 1.480000 at 78.208159 degrees.
Reading of 1.479000 at 79.210945 degrees.
Reading of 1.482000 at 80.213730 degrees.
Reading of 1.482000 at 81.216516 degrees.
Reading of 1.485000 at 83.222087 degrees.
Reading of 1.481000 at 84.224872 degrees.
Reading of 1.482000 at 85.227658 degrees.
Reading of 1.487000 at 86.230443 degrees.
Reading of 1.486000 at 87.233229 degrees.
Reading of 1.498000 at 88.236014 degrees.
Reading of 1.500000 at 89.238800 degrees.
Reading of 1.501000 at 90.241585 degrees.
Reading of 1.519000 at 91.244371 degrees.
Reading of 3.812000 at 93.249942 d

...

Reading of 1.777000 at 61.160805 degrees.
Reading of 1.799000 at 62.163591 degrees.
Reading of 1.825000 at 63.166376 degrees.
Reading of 1.846000 at 64.169162 degrees.
Reading of 1.870000 at 65.171947 degrees.
Reading of 1.906000 at 66.174733 degrees.
Reading of 1.930000 at 67.177518 degrees.
Reading of 1.989000 at 68.180304 degrees.
Reading of 1.948000 at 69.183089 degrees.
Reading of 1.914000 at 70.185875 degrees.
Reading of 1.885000 at 71.188660 degrees.
Reading of 1.838000 at 72.191446 degrees.
Reading of 1.812000 at 74.197017 degrees.
Reading of 1.774000 at 75.199802 degrees.
Reading of 1.753000 at 76.202588 degrees.
Reading of 1.724000 at 77.205374 degrees.
Reading of 1.693000 at 78.208159 degrees.
Reading of 1.679000 at 79.210945 degrees.
Reading of 1.652000 at 80.213730 degrees.
Reading of 1.625000 at 81.216516 degrees.
Reading of 1.608000 at 82.219301 degrees.
Reading of 1.584000 at 83.222087 degrees.
Reading of 1.569000 at 84.224872 degrees.
Reading of 1.548000 at 85.227658 d

Reading of 3.587000 at 120.325151 degrees.
Reading of 3.556000 at 121.327936 degrees.
Reading of 3.549000 at 122.330722 degrees.
Reading of 3.561000 at 123.333507 degrees.
Reading of 3.603000 at 124.336293 degrees.
Reading of 3.606000 at 125.339078 degrees.
Reading of 3.603000 at 126.341864 degrees.
Reading of 3.588000 at 127.344649 degrees.
Reading of 3.620000 at 128.347435 degrees.
Reading of 3.636000 at 129.350220 degrees.
Reading of 3.673000 at 131.355791 degrees.
Reading of 3.669000 at 132.358577 degrees.
Reading of 3.660000 at 133.361362 degrees.
Reading of 3.697000 at 134.364148 degrees.
Reading of 4.309000 at 0.993674 degrees.
Reading of 4.282000 at 1.996460 degrees.
Reading of 4.261000 at 2.999245 degrees.
Reading of 4.236000 at 4.002031 degrees.
Reading of 4.224000 at 5.004816 degrees.
Reading of 4.173000 at 7.010387 degrees.
Reading of 4.127000 at 8.013173 degrees.
Reading of 4.080000 at 9.015958 degrees.
Reading of 4.107000 at 10.018744 degrees.
Reading of 4.055000 at 11.02

Reading of 1.649000 at 25.060527 degrees.
Reading of 1.650000 at 26.063312 degrees.
Reading of 1.648000 at 27.066098 degrees.
Reading of 1.646000 at 28.068883 degrees.
Reading of 1.647000 at 29.071669 degrees.
Reading of 1.650000 at 30.074454 degrees.
Reading of 1.646000 at 32.080025 degrees.
Reading of 1.520000 at 34.085596 degrees.
Reading of 1.521000 at 35.088382 degrees.
Reading of 1.526000 at 36.091167 degrees.
Reading of 1.531000 at 37.093953 degrees.
Reading of 1.540000 at 38.096738 degrees.
Reading of 1.536000 at 39.099524 degrees.
Reading of 1.545000 at 40.102309 degrees.
Reading of 1.552000 at 41.105095 degrees.
Reading of 1.558000 at 42.107880 degrees.
Reading of 1.562000 at 43.110666 degrees.
Reading of 1.567000 at 44.113452 degrees.
Reading of 1.582000 at 45.116237 degrees.
Reading of 1.590000 at 46.119023 degrees.
Reading of 1.603000 at 48.124594 degrees.
Reading of 1.613000 at 49.127379 degrees.
Reading of 1.626000 at 50.130165 degrees.
Reading of 1.634000 at 51.132950 d

Reading of 3.593000 at 114.308438 degrees.
Reading of 3.564000 at 115.311223 degrees.
Reading of 3.571000 at 116.314009 degrees.
Reading of 3.570000 at 117.316794 degrees.
Reading of 3.552000 at 118.319580 degrees.
Reading of 3.571000 at 118.319580 degrees.
Reading of 3.589000 at 119.322365 degrees.
Reading of 3.576000 at 120.325151 degrees.
Reading of 3.562000 at 121.327936 degrees.
Reading of 3.587000 at 122.330722 degrees.
Reading of 3.579000 at 123.333507 degrees.
Reading of 3.604000 at 124.336293 degrees.
Reading of 3.595000 at 125.339078 degrees.
Reading of 3.649000 at 127.344649 degrees.
Reading of 3.605000 at 128.347435 degrees.
Reading of 3.683000 at 129.350220 degrees.
Reading of 3.650000 at 130.353006 degrees.
Reading of 3.701000 at 131.355791 degrees.
Reading of 3.701000 at 132.358577 degrees.
Reading of 3.709000 at 133.361362 degrees.
Reading of 3.701000 at 134.364148 degrees.
Reading of 4.324000 at 1.996460 degrees.
Reading of 4.331000 at 2.999245 degrees.
Reading of 4.24

[array([[2.72900009, 2.08256814],
       [2.77900004, 2.10007005],
       [2.77699995, 2.11757196],
       [2.704     , 2.13507387],
       [2.65400004, 2.15257578],
       [2.59100008, 2.17007769],
       [2.52200007, 2.1875796 ],
       [2.46000004, 2.20508151],
       [2.40499997, 2.22258341],
       [2.3670001 , 2.25758723],
       [2.30900002, 2.27508914],
       [2.2579999 , 2.29259105],
       [2.22399998, 2.31009296],
       [2.19199991, 2.32759487],
       [2.1559999 , 2.34509678],
       [0.34599999, 0.01734289],
       [0.33399999, 0.0348448 ],
       [0.32499999, 0.05234671],
       [0.31900001, 0.06984862],
       [0.315     , 0.08735052],
       [0.31099999, 0.10485243],
       [0.308     , 0.12235434],
       [0.30599999, 0.13985625],
       [0.30599999, 0.15735816],
       [0.308     , 0.17486007],
       [0.31299999, 0.19236198],
       [0.32600001, 0.2273658 ],
       [1.92400002, 0.29737343],
       [1.92700005, 0.33237725],
       [1.94400001, 0.34987916],
       [1

In [12]:
plt.figure(1)
plt.cla()
idx = 3
tmp = ptsHist[idx]
plt.scatter(tmp[:,0],tmp[:,1],0.5)
plt.axis('equal')
plt.show()

hist = fptsHist[idx]

plt.figure(0)
plt.cla()
plt.scatter(hist[:,0],hist[:,1],0.5)
plt.axis('equal')
plt.show()

print(fdistHist.shape)

(215,)


In [14]:
print(fdistHist[0][0,:])

[2.72900009 2.08256814]


In [53]:
idx = 3
prevPt = None
deltas = []
for pt in fptsHist[idx]:
    if prevPt is None:
        prevPt = pt
        continue
    dPt = np.array(pt) - np.array(prevPt)
#     print(dPt)
    deltas.append(dPt)
    
print(" ------------ ")
prevRd = None
dReadings = []
for pt in fdistHist[idx]:
    if prevRd is None:
        prevRd = pt
        continue
    dR = pt - prevRd
#     print(dR)
    dReadings.append(dR)
    
deltas = np.array(deltas)
dReadings = np.array(dReadings)
print(len(deltas),len(dReadings))

 ------------ 
(108, 108)


In [56]:
for i in range(0,len(fdistHist)):
    if i == len(fdistHist):
        break
    dx = fptsHist[idx][i+1,0]-fptsHist[idx][i,0]
    dDist = fdistHist[idx][i+1,0] - fdistHist[idx][i,0]
    dAng = fdistHist[idx][i+1,1] - fdistHist[idx][i,1]
    print("[%d] Point: %s = %.2f" % (i,str(fptsHist[idx][i,:]), dx))
#     print("\tdX, dDist,dAng: %.2f, %.2f, %.2f" % (dx,dDist,dAng))

[0] Point: [-1.28559439  2.38565448] = -0.05
[1] Point: [-1.33351527  2.37412432] = -0.11
[2] Point: [-1.43866197  2.36358637] = -0.00
[3] Point: [-1.44237044  2.27889812] = -0.01
[4] Point: [-1.45345794  2.20986083] = -0.01
[5] Point: [-1.4608874   2.13866957] = 0.00
[6] Point: [-1.46049574  2.05975185] = 0.00
[7] Point: [-1.45839321  1.98232441] = 0.01
[8] Point: [-1.4449405   1.89369237] = -0.01
[9] Point: [-1.45366401  1.83752814] = -0.01
[10] Point: [-1.45960363  1.78010153] = 0.00
[11] Point: [-1.45815895  1.7161809 ] = -0.01
[12] Point: [-1.46484483  1.664127  ] = 0.00
[13] Point: [-1.46207733  1.6035056 ] = 0.01
[14] Point: [-1.45695964  1.54277429] = -0.01
[15] Point: [-1.46485781  1.49773719] = 1.81
[16] Point: [0.34794766 0.00603502] = -0.01
[17] Point: [0.33479666 0.01167065] = -0.01
[18] Point: [0.32655208 0.01710956] = -0.01
[19] Point: [0.32021728 0.02240318] = -0.01
[20] Point: [0.31028651 0.03265405] = -0.00
[21] Point: [0.3056974  0.03759118] = -0.00
[22] Point: [0.30

IndexError: index 109 is out of bounds for axis 0 with size 109

In [47]:
print(len(fdistHist))

215


In [ ]:
for i in range(len(fdistHist[idx])):
    